In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [ ]:
X = [] #skin cancer image
y = [] #mask of skin cancer

def create_train_data():
    DATADIR = "C:/Users/Kritagya Nayyar/Downloads/Data/Training" #local directory for skin cancer original images
    DATADIR2 = "C:/Users/Kritagya Nayyar/Downloads/Data/Truth" #local directory for skin cancer mask images
    IMG_SIZE = 96
    

    for img in os.listdir(DATADIR):
        try:
            img_arr=cv2.imread(os.path.join(DATADIR, img), cv2.IMREAD_GRAYSCALE)
            new_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
            new_arr = new_arr/255.0
            X.append(new_arr)
        except Exception as e:
            pass
    for img in os.listdir(DATADIR2):
        try:        
            truth_img_arr=cv2.imread(os.path.join(DATADIR2, img), cv2.IMREAD_GRAYSCALE)
            truth_new_arr = cv2.resize(truth_img_arr, (IMG_SIZE, IMG_SIZE))
            truth_new_arr = truth_new_arr/255.0
            y.append(truth_new_arr)
        except Exception as e:
            pass
create_train_data()

In [ ]:
X = np.array(X).reshape(-1, 96, 96, 1)
y = np.array(y).reshape(-1, 96, 96, 1)

In [ ]:
np.save("C:/Users/Kritagya Nayyar/Downloads/Data/X.npy",X)
np.save("C:/Users/Kritagya Nayyar/Downloads/Data/y.npy",y)

In [ ]:
X = np.load("C:/Users/Kritagya Nayyar/Downloads/Data/X.npy")
y = np.load("C:/Users/Kritagya Nayyar/Downloads/Data/y.npy")

In [ ]:
def unet(pretrained_weights = None,input_size = (96,96,1)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    #3x3 matric on i'inputs' - 2 times
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #Pooling - 2x2 matrix applied on conv1

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    #3x3 matric on pooled data 
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    #3x3 matrix on convolution applied on pooled data
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #Pooling - 2x2 matrix applied on conv2
    
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    #3x3 matric on pooled data
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    #3x3 matrix on convolution applied on pooled data
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #Pooling - 2x2 matrix applied on conv3

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    #image converted to 512 pixels

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    #up convulation aaplied on conv5 (512 pixels) which is to be superimposed with conv 4 (256 pixels) using 2x2 matrix
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    #3x3 matrix on up6 - convert to 256 pixels
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    #up convulation aaplied on conv6 (256 pixels) which is to be superimposed with conv3 (128 pixels) 
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    #3x3 matrix on up6 - convert to 128 pixels
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    #up convulation aaplied on conv7 (128 pixels) which is to be superimposed with conv2 (64 pixels)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    #3x3 matrix on up6 - convert to 64 pixels
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    #up convulation aaplied on conv8 (64 pixels) which is to be superimposed with conv1 (32 pixels)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    #3x3 matrix on up6 - convert to 32 pixels
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    #conv9 converted to the image of original size under sigmoid activation by 1x1 matrix. The matrix is named conv10

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
#imgs_train,imgs_mask_train = geneTrainNpy("C:/Users/Kritagya Nayyar/Downloads/Data/X.npy","C:/Users/Kritagya Nayyar/Downloads/Data/y.npy")
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit(X, y, batch_size=2, epochs=5, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

In [ ]:
model = model.load_weights('C:/Users/Kritagya Nayyar/Downloads/unet_membrane.hdf5')

In [ ]:
#testing for a singl image
arr1=cv2.imread("C:/Users/Kritagya Nayyar/Downloads/Data/Training/ISIC_0000000.jpg", cv2.IMREAD_GRAYSCALE)
IMG_SIZE = 96
arr = cv2.resize(arr1, (IMG_SIZE, IMG_SIZE))
arr = arr/255.0
print (arr.shape)
arr = arr.reshape(-1, 96, 96, 1)
print (arr.shape)

In [ ]:
#fitting testing image in model
results = model.predict(arr)
print(results.shape)
results = results.reshape(96, 96)
print(results.shape)
print(results)
plt.imshow(results, cmap="gray")
plt.show()